In [153]:
def convert_chord(tu):
    origin = [0,0,0,0,0,0,0,0,0,0,0,0]
    for i in tu:
        origin[i%12]=1
    return tuple(origin)


In [154]:
import numpy as np
# chord = {
#     convert((0,4,7)):'Major',convert((0,3,7)):'minor',
#     convert((0,4,8)):'aug',convert((0,3,6)):'dim',
#     convert((0,4,7,11)):'Major7',convert((0,4,7,10)):'dom7',
#     convert((0,3,7,10)):'minor7',convert((0,3,6,10)):'m7flat5',
#     convert((0,2,7)):'2',convert((0,5,7)):'4',
#     convert((0,2,4,7,10)):'9'
# }
chord = {
    convert_chord((0,4,7)):'C',convert_chord((2,5,9)):'Dm',convert_chord((4,7,11)):'Em',
    convert_chord((0,5,9)):'F',convert_chord((2,7,11)):'G',convert_chord((0,4,9)):'A',
}

chord_np = np.array(chord.keys())

key = {
    0:'C',1:'C-sharp',2:'D',3:'D-sharp',4:'E',5:'F',6:'F-sharp',7:'G',8:'G-sharp',9:'A',10:'A-sharp',11:'B'
} # pitch%12

time = {
    'measure':1,'whole':1,'half':1.0/2,'quarter':1.0/4,'eighth':1.0/8,'16th':1.0/16,'32nd':1.0/32,'64th':1.0/64,
    '128th':1.0/128
}

In [155]:
from bs4 import BeautifulSoup as bs
f = open('E:/downloadmusic2/Alone in my room-test.mscx','rb') # 此首是C小調
music = bs(f.read(),'xml')
f.close()
l = []
dic = {}
for staff in music.select('Score > Staff'):
    time_acc = 0
    for tag in staff.find_all(['Rest','Chord']):
        duTime = time[tag.select('durationType')[0].text]
        time_acc += duTime
        if tag.name == 'Rest':
            l.append((time_acc,0))
        else:
            pitches = tag.select('pitch')
            addlist = [time_acc] +  [int(a.text)-3 for a in pitches]
            l.append(tuple(addlist))
    dic.update({'staff_'+staff.get('id') : l})
    l = []

In [169]:
# 第一步，判斷每個staff，有幾成的比例，使用和弦   by Henry
# 1-1. 目前預設，只要大於70%，就紀錄該staff
# 1-2. pitch只要同時有2音以上，就當作和弦

pitch_count = 0
percent = 0.7
dominant_percent_staff_list = []

for staff in dic:
    for tempo_pitch in dic[staff]:
        if len(tempo_pitch) > 3 :
            pitch_count += 1
    if float(pitch_count) / float(len(dic[staff])) >= percent:
        dominant_percent_staff_list.append(staff)
print dominant_percent_staff_list

#測試區

# for staff in dominant_percent_staff_list:
#     print staff
# print dic['staff_4']
# print type(dic)
# for staff in dic:
#     print staff
# for tempo_pitch in dic[staff]:
#     print tempo_pitch
#     print len(tempo_pitch)
# print pitch_count
# print type(pitch_count)
# print len(dic['staff_2'])
# print type(len(dic['staff_2']))
# if float(pitch_count) / float(len(dic['staff_2'])) >= 0.3:
#     dominant_percent_staff_list.append('staff_2')
#     print dominant_percent_staff_list
print type(dic[staff][-12][1:])
print type(chord_np)
print type(convert_chord(tuple(now)))

[u'staff_2', u'staff_3']
<type 'tuple'>
<type 'numpy.ndarray'>
<type 'tuple'>


In [157]:
import numpy as np
import math
def nearest(nparray, pt):
    n = [math.sqrt(sum((i-pt)**2)) for i in nparray]
    mindis = min(n)
    idx = n.index(mindis)
    return nparray[idx]


In [170]:
chord_dic = {}
pitch_count = 0
percent = 0.7
for staff in dic:
    chord_gogo = []
    chord_gogogo = []
    count = 0
    begin = 0
    
    for tempo_pitch in dic[staff]:
        if len(tempo_pitch) > 3 :
            pitch_count += 1
    if float(pitch_count) / float(len(dic[staff])) >= percent:
        
        while(count < dic[staff][-1][0]):
            while(count < dic[staff][begin][0]):
                count += 1.0/2
                if dic[staff][begin][1] == 0:
                    chord_gogo.append(0)
                else:
                    now = [pit%12 for pit in dic[staff][begin][1:]]
                    closest = nearest(chord_np,convert_chord(tuple(now)))  
                    chord_gogo.append(chord[tuple(closest)])

            begin += 1
        chord_dic.update({staff : chord_gogo})

In [177]:
print len(chord_dic['staff_3'])

450


In [116]:
for ch in chord_dic['staff_2']:

SyntaxError: invalid syntax (<ipython-input-116-8430e92f8c05>, line 1)

In [147]:
c = []
for i in range(1,len(chord_dic['staff_1'])+1):
    if i%8 == 0:
        c.append(chord_dic['staff_1'][i-1])
        print c
        c = []
    else:
        c.append(chord_dic['staff_1'][i-1])


[0, 0, 0, 0, 'C', 'C', 'G', 'G']
['A', 'A', 'Em', 'Em', 'F', 'F', 'C', 'C']
['G', 'G', 'G', 'G', 'C', 'C', 'G', 'G']
['A', 'A', 'Em', 'Em', 'F', 'F', 'C', 'C']
['G', 'G', 'G', 'G', 'F', 'F', 'G', 'G']
['C', 'C', 'C', 'C', 'F', 'F', 'G', 'G']
['C', 'C', 'C', 'C', 'F', 'F', 'G', 'G']
['C', 'F', 'A', 'A', 'F', 'F', 'G', 'G']
['C', 'G', 'A', 'G', 'C', 'G', 'A', 'G']
['C', 'C', 'G', 'G', 'A', 'A', 'Em', 'Em']
[0, 0, 0, 'F', 'A', 0, 'F', 0]
[0, 0, 'C', 'C', 'G', 'G', 'G', 'G']
['C', 'C', 'G', 'G', 'A', 'A', 'Em', 'Em']
['F', 0, 'G', 'C', 'A', 0, 'F', 0]
[0, 0, 'C', 'C', 'G', 'G', 'G', 'G']
['F', 'F', 'G', 'G', 'C', 'C', 'C', 'C']
['F', 'F', 'G', 'G', 'C', 'C', 'C', 'C']
['F', 'F', 'G', 'G', 'C', 'F', 'A', 'A']
['F', 'F', 'G', 'G', 'C', 'G', 'A', 'G']
['C', 'G', 'A', 'G', 'C', 'G', 'A', 'G']
['C', 'G', 'A', 'G', 'F', 'F', 'F', 'F']
['G', 'G', 'G', 0, 'C', 'C', 'G', 'G']
['A', 'A', 'Em', 'Em', 0, 0, 'F', 'C']
[0, 0, 0, 0, 0, 0, 'C', 'C']
['G', 'G', 'G', 'G', 'C', 'C', 'G', 'G']
['A', 'A', 'Em'

In [70]:
chord_gogo = []
count = 0
begin = 0
while(count<l2[-1][0]):
    while(count < l2[begin][0]):
        count += 1.0/2
        if l2[begin][1] == 0:
            chord_gogo.append(0)
        else:
            chord_gogo.append(key[l2[begin][1]%12])
    begin += 1

[5, 1, 3]


In [ ]:
now = [pit%12 for pit in dic[staff][begin][1:]]
                for pitch in now:
                    poss_key = key[pitch]
                    det = [(n-pitch)%12 for n in now]
                    vec = convert_chord(tuple(det))
                    poss_chord = nearest(chord_np,vec)[0]